In [1]:
import pandas as pd
aqi_df=pd.read_csv("D://aqi.csv")
idsp_df= pd.read_csv("D://idsp.csv",encoding='latin1')
popu_df= pd.read_csv("D://population_projection.csv")
vahan_df= pd.read_csv("D://vahan.csv")

In [11]:
# Preprocess dates and add year/month columns
aqi_df['date'] = pd.to_datetime(aqi_df['date'], format='%d-%m-%Y', errors='coerce')
aqi_df['year'] = aqi_df['date'].dt.year
aqi_df['month'] = aqi_df['date'].dt.month  # Create month column to fix KeyError
idsp_df['outbreak_starting_date'] = pd.to_datetime(idsp_df['outbreak_starting_date'], format='%d-%m-%Y', errors='coerce')
idsp_df['year'] = idsp_df['outbreak_starting_date'].dt.year
idsp_df['month'] = idsp_df['outbreak_starting_date'].dt.month
# For vahan and popu, assume month is string or number; convert to int if needed
vahan_df['month'] = pd.to_numeric(vahan_df['month'], errors='coerce')
popu_df['month'] = pd.to_numeric(popu_df['month'], errors='coerce')

In [13]:
# Current date for Q3
current_date = pd.to_datetime('2025-09-15')

In [15]:
# Question 1: Top 5 and bottom 5 areas with highest average AQI (Dec 2024 to May 2025)
start_q1 = pd.to_datetime('2024-12-01')
end_q1 = pd.to_datetime('2025-05-31')
filtered_q1 = aqi_df[(aqi_df['date'] >= start_q1) & (aqi_df['date'] <= end_q1)].dropna(subset=['aqi_value'])
area_counts = filtered_q1.groupby('area').size()
valid_areas = area_counts[area_counts > 0].index
avg_aqi = filtered_q1[filtered_q1['area'].isin(valid_areas)].groupby('area')['aqi_value'].mean().sort_values(ascending=False)
top5_q1 = avg_aqi.head(5)
bottom5_q1 = avg_aqi.sort_values(ascending=True).head(5)
print("Q1 Top 5 Areas by Avg AQI:\n", top5_q1)
print("Q1 Bottom 5 Areas by Avg AQI:\n", bottom5_q1)


Q1 Top 5 Areas by Avg AQI:
 area
Byrnihat       284.194915
Delhi          238.920000
Hajipur        233.666667
Bahadurgarh    226.437500
Gurugram       204.143836
Name: aqi_value, dtype: float64
Q1 Bottom 5 Areas by Avg AQI:
 area
Tirunelveli       33.310078
Palkalaiperur     42.794872
Madikeri          42.951049
Vijayapura        44.328767
Chamarajanagar    44.807692
Name: aqi_value, dtype: float64


In [17]:
# Question 2: Top 2 and bottom 2 prominent pollutants for each Southern state (2022 onward)
southern_states = ['Andhra Pradesh', 'Telangana', 'Karnataka', 'Kerala', 'Tamil Nadu']
filtered_q2 = aqi_df[(aqi_df['year'] >= 2022) & aqi_df['state'].isin(southern_states)]
pollutant_counts = filtered_q2.groupby(['state', 'prominent_pollutants']).size().reset_index(name='count')
for state in southern_states:
    state_data = pollutant_counts[pollutant_counts['state'] == state].sort_values('count', ascending=False)
    top2 = state_data.head(2)['prominent_pollutants'].tolist()
    bottom2 = state_data.tail(2)['prominent_pollutants'].tolist()
    print(f"Q2 {state} Top 2 Pollutants: {top2}")
    print(f"Q2 {state} Bottom 2 Pollutants: {bottom2}")

Q2 Andhra Pradesh Top 2 Pollutants: ['PM10', 'PM2.5']
Q2 Andhra Pradesh Bottom 2 Pollutants: ['PM10,NO2,PM2.5,O3', 'PM10,NO2,O3']
Q2 Telangana Top 2 Pollutants: ['PM2.5,PM10', 'PM10']
Q2 Telangana Bottom 2 Pollutants: ['PM2.5,NO2', 'NO2']
Q2 Karnataka Top 2 Pollutants: ['PM10', 'CO']
Q2 Karnataka Bottom 2 Pollutants: ['NO2,O3', 'SO2,O3']
Q2 Kerala Top 2 Pollutants: ['PM10', 'PM2.5']
Q2 Kerala Bottom 2 Pollutants: ['SO2', 'CO,O3']
Q2 Tamil Nadu Top 2 Pollutants: ['PM10', 'PM2.5']
Q2 Tamil Nadu Bottom 2 Pollutants: ['PM10,NH3,CO', 'PM10,NO2,PM2.5,O3']


In [19]:
# Question 3: AQI improve on weekends vs weekdays in metro cities (last 1 year from max date)
metros = ['Delhi', 'Mumbai', 'Chennai', 'Kolkata', 'Bengaluru', 'Hyderabad', 'Ahmedabad', 'Pune']
max_date = aqi_df['date'].max()
start_q3 = max_date - pd.Timedelta(days=365)
filtered_q3 = aqi_df[(aqi_df['date'] >= start_q3) & (aqi_df['date'] <= max_date) & aqi_df['area'].isin(metros)].copy()  # Added .copy()
filtered_q3.loc[:, 'weekday'] = filtered_q3['date'].dt.weekday
filtered_q3.loc[:, 'is_weekend'] = filtered_q3['weekday'] >= 5
avg_weekday = filtered_q3[~filtered_q3['is_weekend']].groupby('area')['aqi_value'].mean()
avg_weekend = filtered_q3[filtered_q3['is_weekend']].groupby('area')['aqi_value'].mean()
improvement = ((avg_weekday - avg_weekend) / avg_weekday * 100).fillna(0)
print("Q3 Weekday Avg AQI:\n", avg_weekday)
print("Q3 Weekend Avg AQI:\n", avg_weekend)
print("Q3 % Improvement:\n", improvement)

Q3 Weekday Avg AQI:
 area
Ahmedabad    114.716475
Bengaluru     71.896552
Chennai       71.245211
Delhi        208.697318
Hyderabad     77.923372
Kolkata       91.727969
Mumbai        91.049808
Pune         101.954023
Name: aqi_value, dtype: float64
Q3 Weekend Avg AQI:
 area
Ahmedabad    116.038462
Bengaluru     72.384615
Chennai       68.442308
Delhi        198.923077
Hyderabad     79.009615
Kolkata       91.259615
Mumbai        92.653846
Pune         100.846154
Name: aqi_value, dtype: float64
Q3 % Improvement:
 area
Ahmedabad   -1.152395
Bengaluru   -0.678842
Chennai      3.934163
Delhi        4.683453
Hyderabad   -1.393990
Kolkata      0.510590
Mumbai      -1.761715
Pune         1.086636
Name: aqi_value, dtype: float64


In [21]:
# Question 4: Months consistently show worst air quality across top 10 states with high distinct areas
distinct_areas = aqi_df.groupby('state')['area'].nunique().sort_values(ascending=False).head(10)
top_states = distinct_areas.index
filtered_q4 = aqi_df[aqi_df['state'].isin(top_states)]
monthly_avg = filtered_q4.groupby(['state', 'month'])['aqi_value'].mean().reset_index()
worst_months_per_state = monthly_avg.loc[monthly_avg.groupby('state')['aqi_value'].idxmax()]['month']
consistent_months = worst_months_per_state.value_counts().sort_values(ascending=False)
print("Q4 Consistent Worst Months:\n", consistent_months)

Q4 Consistent Worst Months:
 month
11    5
1     3
12    1
2     1
Name: count, dtype: int64


In [23]:
# Question 5: For Bengaluru, days under each air quality category (Mar-May 2025)
start_q5 = pd.to_datetime('2025-03-01')
end_q5 = pd.to_datetime('2025-05-31')
filtered_q5 = aqi_df[(aqi_df['area'] == 'Bengaluru') & (aqi_df['date'] >= start_q5) & (aqi_df['date'] <= end_q5)]
category_counts = filtered_q5['air_quality_status'].value_counts()
print("Q5 Bengaluru Category Counts:\n", category_counts)

Q5 Bengaluru Category Counts:
 air_quality_status
Satisfactory    48
Moderate        13
Name: count, dtype: int64


In [25]:
# Question 6: Severity Mapping - cities with persistent or worsening AQI, count unhealthy+ days
from scipy.stats import linregress
from datetime import datetime
aqi_df['unhealthy'] = aqi_df['aqi_value'] >= 151
unhealthy_days = aqi_df.groupby('area')['unhealthy'].sum()
trends = {}
for area, group in aqi_df.groupby('area'):
    if len(group) < 10: continue
    group = group.sort_values('date')
    x = (group['date'] - group['date'].min()).dt.days.values
    y = group['aqi_value'].values
    slope, _, _, _, _ = linregress(x, y)
    trends[area] = {'slope': slope, 'persistent': group['aqi_value'].mean() > 100}
print("Q6 Trends (sample):\n", dict(list(trends.items())[:5]))
print("Q6 Unhealthy Days (Top 10):\n", unhealthy_days.sort_values(ascending=False).head(10))

Q6 Trends (sample):
 {'Agartala': {'slope': 0.03893017918016549, 'persistent': True}, 'Agra': {'slope': -0.0014232614889237943, 'persistent': False}, 'Ahmedabad': {'slope': 0.011129704812530151, 'persistent': True}, 'Ahmednagar': {'slope': 0.19210841932724798, 'persistent': True}, 'Aizawl': {'slope': 0.013389226699217343, 'persistent': False}}
Q6 Unhealthy Days (Top 10):
 area
Greater Noida    750
Delhi            730
Gurugram         724
Bhiwadi          659
Ghaziabad        657
NOIDA            643
Faridabad        622
Patna            618
Muzaffarnagar    571
Hajipur          560
Name: unhealthy, dtype: int64


In [26]:
# Question 7: Health Impact Correlation - correlate AQI spikes with health events
air_related = ['Chickenpox', 'Fever with Rash', 'Measles', 'Acute Diarrheal Disease']
idsp_filtered = idsp_df[idsp_df['disease_illness_name'].isin(air_related)]
health_monthly = idsp_filtered.groupby(['state', 'year', 'month'])['cases'].sum().reset_index(name='health_events')
aqi_monthly = aqi_df.groupby(['state', 'year', 'month'])['aqi_value'].agg(['mean', lambda x: (x > 200).sum()]).reset_index()
aqi_monthly.columns = ['state', 'year', 'month', 'avg_aqi', 'spikes']
merged = pd.merge(aqi_monthly, health_monthly, on=['state', 'year', 'month'], how='inner')
# Filter groups with enough data for correlation (at least 2 rows and non-zero variance)
merged = merged.groupby('state').filter(lambda g: len(g) >= 2 and g['avg_aqi'].var() > 0 and g['health_events'].var() > 0)
overall_corr = merged['avg_aqi'].corr(merged['health_events']) if not merged.empty else np.nan
state_corrs = merged.groupby('state').apply(
    lambda g: g[['avg_aqi', 'health_events']].corr().iloc[0, 1], include_groups=False
).dropna()
print("Q7 Overall Correlation:", overall_corr)
print("Q7 State Correlations:\n", state_corrs)

Q7 Overall Correlation: -0.14838313305561446
Q7 State Correlations:
 state
Andhra Pradesh      -0.309552
Arunachal Pradesh   -0.244330
Assam               -0.216590
Bihar               -0.538336
Chhattisgarh        -0.471196
Gujarat             -0.217269
Haryana             -0.130042
Himachal Pradesh    -0.075137
Jammu and Kashmir   -0.638977
Jharkhand           -0.169634
Karnataka           -0.153381
Kerala               0.242371
Madhya Pradesh      -0.473881
Maharashtra         -0.088981
Manipur              0.066144
Meghalaya           -0.331755
Mizoram              0.108097
Nagaland            -0.279779
Odisha              -0.297676
Puducherry          -0.288905
Punjab              -0.076286
Rajasthan            0.020170
Sikkim              -1.000000
Tamil Nadu          -0.226025
Telangana           -0.217908
Tripura             -0.017199
Uttar Pradesh       -0.355552
Uttarakhand          0.003424
West Bengal          0.020061
dtype: float64


In [29]:
# Question 8: Demand Triggers - temporal relationship AQI spikes and vehicle growth
vahan_monthly = vahan_df.groupby(['state', 'year', 'month'])['value'].sum().reset_index(name='registrations')
vahan_monthly['growth'] = vahan_monthly.groupby('state')['registrations'].pct_change().fillna(0)
merged_q8 = pd.merge(aqi_monthly, vahan_monthly, on=['state', 'year', 'month'], how='inner')
spike_growth_corr = merged_q8['spikes'].corr(merged_q8['growth'])
print("Q8 Spike-Growth Correlation:", spike_growth_corr)

Q8 Spike-Growth Correlation: nan


In [31]:
# Question 9: Market size proxies using vahan and popu for top states (2025)
popu_2025 = popu_df[(popu_df['year'] == 2025) & (popu_df['gender'] == 'Total')].groupby('state')['value'].sum().reset_index(name='population_thousands')
popu_2025['population'] = popu_2025['population_thousands'] * 1000
vahan_2025 = vahan_df[vahan_df['year'] == 2025].groupby('state')['value'].sum().reset_index(name='total_vehicles')
merged_q9 = pd.merge(vahan_2025, popu_2025, on='state', how='inner')
merged_q9['per_capita_vehicles'] = merged_q9['total_vehicles'] / merged_q9['population']
aqi_2025 = aqi_df[aqi_df['year'] == 2025]
high_days = aqi_2025[aqi_2025['aqi_value'] > 150].groupby('state')['date'].nunique().reset_index(name='high_aqi_days')
merged_q9 = pd.merge(merged_q9, high_days, on='state', how='left').fillna(0)
merged_q9['aqi_burden'] = merged_q9['high_aqi_days'] * merged_q9['population'] / 1e6
merged_q9['households'] = merged_q9['population'] / 4.5
merged_q9['vehicles_per_hh'] = merged_q9['total_vehicles'] / merged_q9['households']
top_states_q9 = merged_q9.sort_values('aqi_burden', ascending=False).head(10)
print("Q9 Top States Proxies:\n", top_states_q9)

Q9 Top States Proxies:
              state  total_vehicles  population_thousands  population  \
19     Maharashtra         1028348                189698   189698000   
31   Uttar Pradesh         1286661                176247   176247000   
33     West Bengal          390232                113481   113481000   
18  Madhya Pradesh          526548                 78047    78047000   
27       Rajasthan          473533                 67110    67110000   
7            Delhi          243260                 66635    66635000   
14       Karnataka          600589                 93769    93769000   
9          Gujarat          570097                109932   109932000   
29      Tamil Nadu          695868                127022   127022000   
4            Bihar          443359                 48968    48968000   

    per_capita_vehicles  high_aqi_days  aqi_burden    households  \
19             0.005421          101.0   19159.498  4.215511e+07   
31             0.007300          104.0   18329.

In [ ]:
End